Dependencies

In [ ]:
!pip install datasets
!pip install rapidfuzz
!python -m spacy download en_core_web_sm

import nltk
nltk.download('punkt')

Imports

In [2]:
from datasets import load_dataset
from nltk import word_tokenize
from bs4 import BeautifulSoup
from gensim.models import Word2Vec
import pandas as pd
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from rapidfuzz import fuzz

from collections import defaultdict
from typing import List, Dict, Set, Tuple
import random
import html
import re

Using a news dataset that is mostly used for language modelling tasks (takes ~10 mins to download). While I could have used an NER dataset and train my own NER model, I decide to use an existing statistical one.

In [3]:
df_cc_news = load_dataset('cc_news', split="train").to_pandas()

Dataset cc_news downloaded and prepared to /root/.cache/huggingface/datasets/cc_news/plain_text/1.0.0/6cdde8d7fdaae3e50fb61b5d08d5387c2f0bbea1ee68755ef954af539a6a3a1b. Subsequent calls will reuse this data.


Given the nature of the task, heavy text preprocessing is probably not necessary, especially the rare word removal (some entities might appear only once or twice). Moreover, in order for the Regex rules to optimally work, the text should probably remain in the original form. 

In [4]:
def preprocess(corpus: List[str]) -> (List[List[str]], List[str]):
    word_counts = defaultdict(int)
    tokenized_corpus = []
    for doc in corpus:
        # Remove HTML entities/tags
        doc = BeautifulSoup(html.unescape(doc), 'html').get_text()
        # Work tokenization
        tokenized_doc = word_tokenize(doc)
        tokenized_corpus.append(tokenized_doc)

        # Could have considered other preprocessing steps too, such as lowercasing, punctuation removal or stop word or rare word removal, though not really necessary for NER
        for token in tokenized_doc:
            word_counts[token] += 1
    
    removed_words = set()
    for word in word_counts:
        if word_counts[word] < 2:
            # print(word, word_counts[word])

            # Remove words occuring only once
            removed_words.add(word)
    
    tokenized_corpus = [[token for token in tokenized_doc if token not in removed_words] for tokenized_doc in tokenized_corpus]
    preprocessed_corpus = [' '.join([token for token in tokenized_doc if token not in removed_words]) for tokenized_doc in tokenized_corpus]

    return tokenized_corpus, preprocessed_corpus

A Word2Vec model is trained on the dataset to extract word embeddings (could've also considered FastText). Ideally, model should be trained on the whole dataset and not on the sample used in this notebook, however, memory constraints don't allow much scaling.

In [5]:
seed = 10
corpus = df_cc_news['text'].sample(5000, random_state=seed)
tokenized_corpus, preprocessed_corpus = preprocess(corpus)
model = Word2Vec(sentences=tokenized_corpus, size=100, window=5, min_count=2)

Spacy's stastistical NER is used to extract Company names (ORG) and Locations (GPE) from the corpus. Machine learned models could've also been used, such as [pretrained BERT-based](https://huggingface.co/dslim/bert-base-NER) ones. For the Company addresses, a Regex rule of the following form is used:

((address number)? (Adress_name)+) ([Postcode](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/488478/Bulk_Data_Transfer_-_additional_validation_valid_from_12_November_2015.pdf))

Making the address rule so explicit means that we will be getting far fewer false positives, however the true positives are also going to be less. Ways of relaxing the rule would be to make the postcode optional, though for the scope of this assessment and given that no explicit dataset is given I decided precision was more important than recall. A similar approach was taken for the serial numbers:

(alphanumberic_first_part)/(alphanumberic_second_part)(/alphanumberic_next_part)*

In [6]:
nlp = spacy.load("en_core_web_sm", exlude=['tok2vec', 'tagger', 'parser', 'senter', 'attribute_ruler'])

In [7]:
def entity_extractor(nlp, preprocessed_corpus: List[str], corpus: List[str]) -> Dict:
    ents = ['ORG', 'GPE']
    address_key = 'ADD'
    serial_key = 'SER'
    entities = defaultdict(set)

    for doc in preprocessed_corpus:
        doc_parsed = nlp(doc)

        for ent in doc_parsed.ents:
            if ent.label_ in ents:
                entities[ent.label_].add(ent.text)

    address_rule = r'\b(([0-9]{1,3} )?[A-Z][A-Za-z]{1,} ([A-Z][A-Za-z]{2,}(,| )*){0,4})((([A-Za-z][0-9]{1,2})|(([A-Za-z][A-Ha-hJ-Yj-y][0-9]{1,2})|(([A-Za-z][0-9][A-Za-z])|([A-Za-z][A-Ha-hJ-Yj-y][0-9][A-Za-z]?))))\s?[0-9][A-Za-z]{2})\b'
    serial_number_rule = r'\b(?=.*\d)(?=.*[A-Z])([A-Z0-9]{2,}[\\\/|])([A-Z0-9]{2,})([\\\/|][A-Z0-9]{2,})*\b'

    # Apply Regex rules to the unprocessed corpus
    for doc in corpus:

        # Apply address Regex
        matched_adds = re.finditer(address_rule, doc)
        for m in matched_adds:
            entities[address_key].add(m.group())

        # Apply Serial number Regex
        matched_serial = re.finditer(serial_number_rule, doc)
        for m in matched_serial:
            # Discard those not containing at least one number and one alpha (should have been matched by the Regex)
            has_alpha = any(c.isalpha() for c in m.group())
            has_number = any(c.isnumeric() for c in m.group())

            if has_alpha and has_number:
                entities[serial_key].add(m.group())
        
    
    return entities

In [8]:
entities = entity_extractor(nlp, preprocessed_corpus, corpus)

Different systems for recognising the entities used:


*   Company names: ORG entities matched by spacy's NER
*   Addresses: Regex used to capture addresses
*   Serial numbers: Regex used to capture serial numbers
*   Locations: GPE entities matched by spacy's NER



In [9]:
entities['SER']

{'00/HR',
 '10G/25G/40G',
 '1172/JCI96939',
 '32GB/64GB',
 '365/CRM',
 '3G/GPRS',
 '3GB/32GB',
 '3GB/4GB',
 '4GB/32GB',
 '4GB/64GB',
 'AWDM/0444/18',
 'AWDM/1834/17',
 'CRM/365',
 'D402/D403',
 'DS/2DS/3DS',
 'H2/18',
 'MA/4216774',
 'Q4/17'}

In [10]:
entities['ADD']

{'16 Church Street, Bromsgrove, B61 8DN',
 '160 Walmley Road, Sutton Coldfield, B76 2QA',
 '30 Clifton Way, Cambridge CB1 7DY',
 '30 North Colonnade London E14 5GN',
 '33 Chaseville Park Road, London n21 1PQ',
 '7 Pancras Square, London, N1C 4AG',
 'Balleny Green, Little Hay, Shenstone, WS14 0QB',
 'Bedford SquareEX1 1QA',
 'Butterley Station, Ripley DE5 3QZ',
 'Cambridge CB2 1PG',
 'Cambridge CB2 3QN',
 'Cambridge CB5 8AQ',
 'Mayflower Steps, The Barbican, PL1 2LR',
 'Mill Lane, Granta Place, Cambridge CB2 1RT',
 'Moto X4 4GB',
 'Super Puma EC225LP',
 'The M840TR',
 'The Watermill, Newnham Road, Cambridge CB3 9EY'}

One approach of clustering the different entities on each entity group would be to use some similariy metric (e.g embedding cosine distance or edit distance), however I noticed that the distribution of cosine similarities and edit distances among different entities varies. Moreover, embedding similarity requires a model trained on huge datasets in order to be relatively accurate, and edit distance might be more applicable to some entity groups (addresses that e.g might belong to same street but different number) than others. Hence, a weighted mean of the 2 can be used as a similarity metric 

In [17]:
def get_cosine_similarities(entities_set: Set[str], entity_type:str, model) -> Tuple[List[List[float]], List[str]]:
    scores = []

    entities_embs = []
    entities_kept = []
    for ent in entities_set:
        emb = []
        for token in word_tokenize(ent):
            # Use word embeddings if word found in word2vec model vocabulary
            if token in model.wv.vocab:
                emb.append(model[token])
        
        # If no word of the entity is found in the word2vec model, discard entity
        if len(emb):
            entities_kept.append(ent)

            average_emb = np.sum(emb, axis=0) / len(emb)
            entities_embs.append(average_emb)

    # entities_embs = np.array(entities_embs)
    cosine_similarities = cosine_similarity(X=entities_embs)

    print(f'Entities of type {entity_type} kept: {len(entities_kept)}/{len(entities_set)}')
    return (cosine_similarities, entities_kept)

def get_edit_distances(entities_kept: List[str]) -> List[List[float]]:
    edit_distances = np.array([[fuzz.token_sort_ratio(ent_i, ent_j) for ent_j in entities_kept] for ent_i in entities_kept])
    return edit_distances

def plot_similarities_histogram(similarities: List[float], entity:str, is_edit_distance: bool) -> None:
    title =  "Edit distance" if is_edit_distance else "Embeddings' similarity"
    counts, bins = np.histogram(similarities)
    plt.hist(bins[:-1], bins, weights=counts)
    plt.title(f"{title} for `{entity}`")

def get_close_entities(entity: str, entities_kept: List[str], similarities:List[float], std: float=1) -> List[str]:
    mean_similarity = similarities.mean()
    std_similarity = similarities.std()

    idx_similar = np.where(similarities > mean_similarity + std*std_similarity)[0]
    entities_similar = [entities_kept[int(id)] for id in idx_similar]
    
    # print(idx.shape, len(entities_similar))
    return entities_similar

def get_similarities(entities_set: Set[str], entity_type: str, embedding_model, config: Dict):
    cosine_similarities, entities_kept = get_cosine_similarities(entities_set, entity_type, embedding_model)
    edit_distances = get_edit_distances(entities_kept)

    weighted_similarities = cosine_similarities*config['emb'] + edit_distances*config['edit']/100
    return weighted_similarities, cosine_similarities, edit_distances, entities_kept

def get_similar_entities(entities_set: Set[str], entity_type: str, embedding_model, config: Dict, verbose:bool=True):
    cosine_similarities, entities_kept = get_cosine_similarities(entities_set, entity_type, embedding_model)
    edit_distances = get_edit_distances(entities_kept)
    
    similar_entities = defaultdict(list)
    
    for i, entity in enumerate(entities_kept):
        
        close_emb = get_close_entities(entity, entities_kept, cosine_similarities[i], std=config['std_emb'])
        close_edit = get_close_entities(entity, entities_kept, edit_distances[i], std=config['std_edit'])

        close_overall = list(set(close_emb).intersection(close_edit))
        # Remove self
        if entity in close_overall:
            close_overall.remove(entity)

        if len(close_overall):
            
            # Create group
            similar_entities[entity] += close_overall

            if verbose:
                print(f'{entity} is similar to:')
                print(close_overall, '\n')
    

    return similar_entities, entities_kept

The following cell extracts similar entities for each entity within a specific entity group

In [12]:
ent = 'GPE'
# Sensitivity of deviation of embeddings' similarity and edit distance
config = {
    'std_emb': 1,
    'std_edit': 4
}

similar_entities = get_similar_entities(entities[ent], ent, model, config=config)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


Streaming output truncated to the last 5000 lines.
["South Africa ''", "South Africa '", 'South Africa'] 

Jenkins is similar to:
['Denis'] 

Citi is similar to:
['CITY'] 

Fultz is similar to:
['Blitz'] 

Placer County is similar to:
['Pulaski County', 'Lee County', 'Price County', 'Polk County', 'Erie County', 'Lake County', 'Kane County', 'Pike County', 'Mercer County', 'Lancaster County', 'Napa County'] 

The United States Coast Guard is similar to:
['Florida United States', 'the United States Merchant', 'the United States Marine Corps', 'United States District Court', "the United States '", 'the United Arab Emirates', 'United States of America', 'the United Counties League', 'the United States of America', 'the United States', 'the Unites States', 'The United Arab Emirates', 'the United States Military Academy', 'The United States'] 

Cache Creek is similar to:
['Buck Creek', 'Coconut Creek'] 

New Haven is similar to:
['New Armenia', 'Haven', 'Armenia New', 'New Athens', 'New Ham

In [13]:
ent = 'ADD'
# Sensitivity of deviation of embeddings' similarity and edit distance
config = {
    'std_emb': 1,
    'std_edit': 1
}

similar_entities = get_similar_entities(entities[ent], ent, model, config=config)

Entities of type ADD kept: 18/18
Cambridge CB5 8AQ is similar to:
['Cambridge CB2 3QN', 'Cambridge CB2 1PG'] 

Cambridge CB2 3QN is similar to:
['Cambridge CB5 8AQ', 'Cambridge CB2 1PG'] 

Cambridge CB2 1PG is similar to:
['Cambridge CB5 8AQ', 'Cambridge CB2 3QN'] 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [14]:
ent = 'ORG'
# Sensitivity of deviation of embeddings' similarity and edit distance
config = {
    'std_emb': 1,
    'std_edit': 4
}

# Using a sample of the ORG entities because the edit distance calculator makes the computer run out of ram
similar_entities = get_similar_entities(set(list(entities[ent])[:5000]), ent, model, config=config)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


Streaming output truncated to the last 5000 lines.
['Clark Community College'] 

The Canadian Press KINGSTON is similar to:
['The Canadian Press'] 

Simon Fraser University is similar to:
['Northeastern University', 'National University'] 

Priyanka is similar to:
['Priyanka Chopra', 'PA Ryanair', 'Ryanair', 'Rink'] 

DuPont is similar to:
['Spot', 'Ogden Point', 'Support', 'POINT'] 

Carlyle is similar to:
['Lyles'] 

West Bank is similar to:
['Left Bank', 'Derwent Bank', 'Scotia Bank'] 

Saint James Baptist Church is similar to:
['Rocky Creek Baptist Church'] 

the Corrections Department is similar to:
['The Wisconsin Department of Corrections', 'The Department of Corrections'] 

Whittaker is similar to:
['Walker', 'RBC Heritage'] 

University of Wisconsin is similar to:
['University of Utah', 'the University of Wisconsin', 'the University of Virginia', 'the University of Washington', 'the University of Vienna', 'the University of Warwick'] 

Middleton is similar to:
['Middlesbrough'

A weighted matrix of embedding similarities and edit distances is calculated. Clustering algorithms are then applied to the computed, weighted similarity matrix. Density based (DBSCAN) or Hierarchical (Agglomerative) clustering algorithms should be a decent option, however the results aren't that good.

In [24]:
ent = 'ADD'
# Weigh embeddings' similarity and edit distance
config = {
    'emb': 0.75,
    'edit': 0.25
}

weighted_similarities, cosine_similarities, edit_distances, entities_kept = get_similarities(entities[ent], ent, model, config=config)

dbs = DBSCAN(eps=0.15, min_samples=1, metric='precomputed').fit(weighted_similarities)
dbs_clusters = defaultdict(list)

for i, label in enumerate(dbs.labels_):
    dbs_clusters[label].append(entities_kept[i])

dbs_clusters

Entities of type ADD kept: 18/18


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


defaultdict(list,
            {-1: ['Cambridge CB5 8AQ',
              'The Watermill, Newnham Road, Cambridge CB3 9EY',
              'Cambridge CB2 3QN',
              'Super Puma EC225LP',
              'Cambridge CB2 1PG',
              '16 Church Street, Bromsgrove, B61 8DN',
              '30 North Colonnade London E14 5GN',
              'Mayflower Steps, The Barbican, PL1 2LR',
              'Bedford SquareEX1 1QA'],
             0: ['Mill Lane, Granta Place, Cambridge CB2 1RT',
              '7 Pancras Square, London, N1C 4AG',
              'The M840TR',
              '30 Clifton Way, Cambridge CB1 7DY',
              '160 Walmley Road, Sutton Coldfield, B76 2QA',
              'Butterley Station, Ripley DE5 3QZ',
              'Balleny Green, Little Hay, Shenstone, WS14 0QB',
              '33 Chaseville Park Road, London n21 1PQ',
              'Moto X4 4GB']})

In [26]:
agg = AgglomerativeClustering(affinity='precomputed', linkage='complete').fit(weighted_similarities)
agg_clusters = defaultdict(list)

for i, label in enumerate(agg.labels_):
    agg_clusters[label].append(list(entities[ent])[i])

agg_clusters

defaultdict(list,
            {0: ['Cambridge CB5 8AQ',
              'Mill Lane, Granta Place, Cambridge CB2 1RT',
              '7 Pancras Square, London, N1C 4AG',
              'The M840TR',
              '160 Walmley Road, Sutton Coldfield, B76 2QA',
              'Balleny Green, Little Hay, Shenstone, WS14 0QB',
              'Super Puma EC225LP',
              'Cambridge CB2 1PG',
              '16 Church Street, Bromsgrove, B61 8DN',
              'Mayflower Steps, The Barbican, PL1 2LR',
              'Bedford SquareEX1 1QA',
              '33 Chaseville Park Road, London n21 1PQ',
              'Moto X4 4GB'],
             1: ['The Watermill, Newnham Road, Cambridge CB3 9EY',
              '30 Clifton Way, Cambridge CB1 7DY',
              'Butterley Station, Ripley DE5 3QZ',
              'Cambridge CB2 3QN',
              '30 North Colonnade London E14 5GN']})

This is an admittedly naive approach to tackle the problem. Ideally, information mined from a knowledge base should be used in order to tackle the problem as an NEL task and link entities together (e.g. using Spotlight of DBpedia). However, the problem can quickly get much more complicated, as many entities might not be having a relevant match (or even one at all). That also depends on the effiency of the NER tool used in the first place.

